In [14]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import threading

film_budget = [None] * 1000

In [15]:
def get_movie_budget(url,i):
    response = requests.get(url)
    data = BeautifulSoup(response.text,'html.parser')
    budget = data.find('div',{'data-testid':'title-boxoffice-section'})
    if budget is None:
        return ('Not Found')
    else:
        budget = budget.find_all('label',{'class':'ipc-metadata-list-item__list-content-item'})
        if budget is not None:
            film_budget[i] = budget[0].text.split(' ')[0]
    response.close()

In [16]:
def get_movie_titles(data):
    movie_title_temp = data.find_all('h3',{'class':"lister-item-header"})
    movie_title = []
    movie_link = []
    for tag in movie_title_temp:
        a_tag = tag.find('a').get('href')
        a_tag = 'https://www.imdb.com' + str(a_tag)
        movie_link.append(a_tag)
        title = tag.find('a').text
        movie_title.append(title)
    return movie_title,movie_link

In [17]:
def get_movie_genre(data):
    movie_genre_temp = data.find_all('div',{'class':"lister-item-content"})
    movie_genre = []
    for detail_tag in movie_genre_temp:
        genre_tag = detail_tag.find('span',{'class':'genre'})
        if genre_tag:
            movie_genre.append(genre_tag.get_text().strip())
        else:
            movie_genre.append('NA')
    return movie_genre

In [18]:
def get_movie_duration(data):
    movie_duration_temp = data.find_all('span',{'class':'runtime'})
    movie_duration = []
    for tag in movie_duration_temp:
        movie_duration.append(tag.text[:-4])
    return movie_duration

In [19]:
def get_movie_rating(data):
    movie_rating_temp = data.find_all('div',{'class':"inline-block ratings-imdb-rating"})
    movie_rating = []
    for tag in movie_rating_temp:
        movie_rating.append(tag.get_text().strip())
    return movie_rating

In [20]:
def get_movie_year(data):
    movie_year_temp = data.find_all('span',{'class':"lister-item-year text-muted unbold"})
    movie_year = []
    for tag in movie_year_temp:
        movie_year.append(tag.get_text().strip()[-5:-1])
    return movie_year

In [21]:
def get_movie_cast(data):
    movie_cast_temp = data.find_all('div',{'class':'lister-item-content'})
    movie_cast = []
    for tag in movie_cast_temp:
        cast_tag_temp = tag.find_all('a')
        cast_tag = []
        for temp_tag in cast_tag_temp:
            cast_tag.append(temp_tag.text)
        movie_cast.append(cast_tag[12::])
    return movie_cast

In [22]:
def get_movie_certification(data):
    movie_cert_temp = data.find_all('div',{'class':"lister-item-content"})
    movie_cert = []
    for detail_tag in movie_cert_temp:
        certification_tag = detail_tag.find('span',{'class':'certificate'})
        if certification_tag:
            movie_cert.append(certification_tag.text)
        else:
            movie_cert.append('NA')
    return movie_cert

In [23]:
def get_movie_gross(data):
    movie_gross_temp = data.find_all('p',{'class':"sort-num_votes-visible"})
    movie_gross = []
    for detail_tag in movie_gross_temp:
        gross_tag = []
        gross_temp_tag = detail_tag.find_all('span',{'name':'nv'})
        for temp_tag in gross_temp_tag:
            gross_tag.append(temp_tag.text)
        if len(gross_tag) >=2 :
            movie_gross.append(gross_tag[1])
        else:
            movie_gross.append('NA')
    return movie_gross

In [24]:
def get_movie_votes(data):
    movie_votes_temp = data.find_all('p',{'class':"sort-num_votes-visible"})
    movie_votes = []
    for detail_tag in movie_votes_temp:
        votes_tag = detail_tag.find_all('span',{'name':'nv'})
        if votes_tag:
            movie_votes.append(votes_tag[0].text)
        else:
            movie_votes.append('NA')
    return movie_votes

In [25]:
def get_data_from_imdb():
    global film_budget
    complete_data = {
        'title' :[],
        'genre' :[],
        'duration' :[],
        'rating' :[],
        'year' :[],
        'director' :[],
        'major cast' :[],
        '2nd major cast' :[],
        'certification' :[],
        'gross' :[],
        'votes' :[],
        'link' :[],
        'budget' :[]
    }
    for i in range(1,2000,100):
        try:
            url = 'https://www.imdb.com/search/title/?groups=top_1000&sort=user_rating,desc&count=100&start='+str(i)+'&ref_=adv_next'
            response = requests.get(url)
        except:
            break
        if response.status_code != 200:
            break
        data = BeautifulSoup(response.text,'html.parser')
        
        movie_title,movie_link = get_movie_titles(data)
        movie_genre = get_movie_genre(data)
        movie_duration = get_movie_duration(data)
        movie_rating = get_movie_rating(data)
        movie_year = get_movie_year(data)
        movie_cast = get_movie_cast(data)
        movie_certification = get_movie_certification(data)
        movie_gross = get_movie_gross(data)
        movie_votes = get_movie_votes(data)
        
        for i in range(len(movie_title)):
            complete_data['title'].append(str(movie_title[i]))
            complete_data['year'].append(int(movie_year[i]))
            complete_data['duration'].append(int(movie_duration[i]))
            complete_data['genre'].append(movie_genre[i])
            complete_data['certification'].append(str(movie_certification[i]))
            complete_data['rating'].append(float(movie_rating[i]))
            complete_data['gross'].append(movie_gross[i])
            complete_data['votes'].append(movie_votes[i])
            complete_data['director'].append(str(movie_cast[i][0]))
            complete_data['major cast'].append(str(movie_cast[i][1]))
            complete_data['2nd major cast'].append(str(movie_cast[i][2]))
            complete_data['link'].append(movie_link[i])
    
    response.close()
    '''thread_list = []'''
    for i in range(len(complete_data['link'])):
        if complete_data['link'][i] is None:
            continue
        else:
            get_movie_budget(str(complete_data['link'][i]),i)
            '''th = threading.Thread(target=get_movie_budget,args=(str(complete_data['link'][i]),i))'''
            '''thread_list.append(th)
            th.start()
        if(len(thread_list)>8):
            for th in thread_list:
                th.join()
    
    for th in thread_list:
        th.join()'''
        
    complete_data['budget'] = film_budget
    return complete_data

In [26]:
data = get_data_from_imdb()
df = pd.DataFrame(data)
#Final Column without NA and #Num values

IndexError: list index out of range

In [ ]:
df

In [ ]:
df.info()

In [ ]:
df = df[df['gross']!='NA']
df = df[~df.gross.str.match('\#')]
df = df[df.budget.str.match('\$')]

In [ ]:
df['budget'] = df['budget'].str[1:]

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df['budget'] = df['budget'].str.replace(',','')
df['budget'] = df['budget'].apply(pd.to_numeric)

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df.to_csv('data.csv')

In [ ]:
df.describe()